# Intro to Deep Learning Homework 5 & 6

## Problem 2 ONLY

**Nathan Inkawhich**

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Nathan Inkawhich

## Problem 2:  Recurrent Neural Networks (30 points)

In [46]:
from urllib.request import urlretrieve
import os
import numpy as np
import h5py
import string
import random
import tensorflow as tf

### Download and format word embeddings

In [47]:
# Word vectors
if not os.path.isfile('mini.h5'):
    print("Downloading Conceptnet Numberbatch word embeddings...")
    conceptnet_url = 'http://conceptnet.s3.amazonaws.com/precomputed-data/2016/numberbatch/17.06/mini.h5'
    urlretrieve(conceptnet_url, 'mini.h5')
    
# Decode file
with h5py.File('mini.h5', 'r') as f:
    all_words = [word.decode('utf-8') for word in f['mat']['axis1'][:]]
    all_embeddings = f['mat']['block0_values'][:]
    
# Extract English words
english_words = [word[6:] for word in all_words if word.startswith('/c/en/')]
english_word_indices = [i for i, word in enumerate(all_words) if word.startswith('/c/en/')]
english_embedddings = all_embeddings[english_word_indices]

# Normalize Embeddings to unit circle
norms = np.linalg.norm(english_embedddings, axis=1)
normalized_embeddings = english_embedddings.astype('float32') / norms.astype('float32').reshape([-1, 1])

# Create LUT
index = {word: i for i, word in enumerate(english_words)}

In [48]:
def similarity_score(w1, w2):
    score = np.dot(normalized_embeddings[index[w1], :], normalized_embeddings[index[w2], :])
    return score

def print_similarity(w1,w2):
    try:
        print('{0}\t{1}\t'.format(w1,w2), \
          similarity_score('{}'.format(w1), '{}'.format(w2)))
    except:
        print('One of the words is not in the dictionary.')
    return None

In [49]:
# A word is as similar with itself as possible:
print('cat\tcat\t', similarity_score('cat', 'cat'))
# Closely related words still get high scores:
print('cat\tfeline\t', similarity_score('cat', 'feline'))
print('cat\tdog\t', similarity_score('cat', 'dog'))
# Unrelated words, not so much
print('cat\tmoo\t', similarity_score('cat', 'moo'))
print('cat\tfreeze\t', similarity_score('cat', 'freeze'))

cat	cat	 1.0000001
cat	feline	 0.8199548
cat	dog	 0.590724
cat	moo	 0.0039538303
cat	freeze	 -0.030225191


### Prepare movie dataset

In [50]:
remove_punct=str.maketrans('','',string.punctuation)

# This function converts a line of our data file into
# a tuple (x, y), where x is 300-dimensional representation
# of the words in a review, and y is its label.
def convert_line_to_example(line):
    # Pull out the first character: that's our label (0 or 1)
    y = int(line[0])
    # Split the line into words using Python's split() function
    words = line[2:].translate(remove_punct).lower().split()
    # Look up the embeddings of each word, ignoring words not
    # in our pretrained vocabulary.
    embeddings = [normalized_embeddings[index[w]] for w in words
                  if w in index]
    # Take the mean of the embeddings
    x = np.mean(np.vstack(embeddings), axis=0)
    return {'x': x, 'y': y, 'w':embeddings}

# Apply the function to each line in the file.
enc = 'utf-8' # This is necessary from within the singularity shell

### Problem 2.1: Train an MLP off of the average word embedding to predict sentiment (as done in class) but optimize the network settings to maximize performance

#### Format Dataset

In [60]:
### Choose Dataset
with open("Data/movie-simple.txt", "r", encoding=enc) as f:
    dataset = [convert_line_to_example(l) for l in f.readlines()]  
#with open("Data/movie-pang02.txt", "r",encoding=enc) as f:
#    dataset = [convert_line_to_example(l) for l in f.readlines()]

print("Length of Dataset: ",len(dataset))
# Shuffle full dataset
random.shuffle(dataset)

# Split full dataset into train/test splits
batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

Length of Dataset:  1411


#### Build MLP

In [61]:
### Configs
num_hidden_L1 = 100
num_hidden_L2 = 100
learning_rate = .01
num_epochs = 2000

# Clear all old tf graphs
tf.reset_default_graph()

# Placeholders for input
X = tf.placeholder(tf.float32, [None, 300]) # Word embedding size = 300
y = tf.placeholder(tf.float32, [None, 1]) # Binary classification output: "good" or "bad"

# Three-layer MLP
h1 = tf.layers.dense(X, num_hidden_L1, tf.nn.relu)
h2 = tf.layers.dense(h1, num_hidden_L2, tf.nn.relu)
logits = tf.layers.dense(h2, 1)
probabilities = tf.sigmoid(logits)

# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(logits)), y), tf.float32))

# Training
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Initialization of variables
initialize_all = tf.global_variables_initializer()

#### Train MLP

In [62]:
sess = tf.Session()
sess.run(initialize_all)
for epoch in range(num_epochs):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = [sample['x'] for sample in data]
        labels  = [sample['y'] for sample in data]
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
    if epoch % 50 == 0:
        print("Epoch", epoch, "Loss", l, "Acc", acc)
    random.shuffle(train)

# Evaluate on test set
test_reviews = [sample['x'] for sample in test]
test_labels  = [sample['y'] for sample in test]
test_labels = np.array(test_labels).reshape([-1, 1])
acc = sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
print("Final test accuracy:", acc)

Epoch 0 Loss 0.69180435 Acc 0.59
Epoch 50 Loss 0.6843544 Acc 0.53
Epoch 100 Loss 0.6723387 Acc 0.56
Epoch 150 Loss 0.66936374 Acc 0.53
Epoch 200 Loss 0.64258176 Acc 0.62
Epoch 250 Loss 0.64455914 Acc 0.61
Epoch 300 Loss 0.6015594 Acc 0.67
Epoch 350 Loss 0.5527072 Acc 0.86
Epoch 400 Loss 0.5074774 Acc 0.86
Epoch 450 Loss 0.47337112 Acc 0.81
Epoch 500 Loss 0.3932708 Acc 0.87
Epoch 550 Loss 0.25771186 Acc 0.94
Epoch 600 Loss 0.25898987 Acc 0.91
Epoch 650 Loss 0.22260652 Acc 0.94
Epoch 700 Loss 0.23537724 Acc 0.91
Epoch 750 Loss 0.18386932 Acc 0.96
Epoch 800 Loss 0.15567163 Acc 0.98
Epoch 850 Loss 0.1284622 Acc 0.96
Epoch 900 Loss 0.13323972 Acc 0.96
Epoch 950 Loss 0.10479845 Acc 0.99
Epoch 1000 Loss 0.14932638 Acc 0.91
Epoch 1050 Loss 0.1106052 Acc 0.99
Epoch 1100 Loss 0.1391635 Acc 0.94
Epoch 1150 Loss 0.0989832 Acc 0.98
Epoch 1200 Loss 0.06615176 Acc 0.99
Epoch 1250 Loss 0.07240047 Acc 0.98
Epoch 1300 Loss 0.07415715 Acc 0.97
Epoch 1350 Loss 0.09513224 Acc 0.97
Epoch 1400 Loss 0.0749141

In [63]:
sess.close()

### Problem 2.2:  Train a RNN from the word embeddings to predict sentiment (as done in class) and optimize the network settings to maximize performance

#### Format Dataset

In [18]:
### Choose Dataset
with open("Data/movie-simple.txt", "r", encoding=enc) as f:
    dataset = [convert_line_to_example(l) for l in f.readlines()]  
#with open("Data/movie-pang02.txt", "r",encoding=enc) as f:
#    dataset = [convert_line_to_example(l) for l in f.readlines()]

print("Length of Dataset: ",len(dataset))  
random.shuffle(dataset)
batch_size = 1
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

Length of Dataset:  1411


#### Build RNN Model

In [19]:
# Clear old tf stuff
tf.reset_default_graph()

# Configs
n_steps = None
n_inputs = 300
n_neurons = 100
num_epochs = 200

# Input placeholders
X= tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y= tf.placeholder(tf.float32, [None, 1])

# Build RNN
basic_cell = tf.contrib.rnn.BasicRNNCell(n_neurons,activation=tf.nn.tanh)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
last_cell_output=outputs[:,-1,:]
y_=tf.layers.dense(last_cell_output,1)

# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(y_)), y), tf.float32))

# Training
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

#### Train RNN

Note, the batch size here is one because we train on all words from a single review then update the parameters. Thus, training this RNN is much slower than the MLP above which has batch size of 100, so we train the RNN for less epochs.

In [20]:
initialize_all = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initialize_all)
l_ma=.74
acc_ma=.5
for epoch in range(num_epochs):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = np.array([sample['w'] for sample in data]).reshape([1,-1,300])
        labels  = np.array([sample['y'] for sample in data]).reshape([1,1])
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        l_ma=.99*l_ma+(.01)*l
        acc_ma=.99*acc_ma+(.01)*acc
        #if (batch+1) % 100 == 0:
        #    print("batch", batch, "Loss", l_ma, "Acc", acc_ma)
    if epoch % 10 == 0:
        print("Epoch", epoch, "Loss", l_ma, "Acc", acc_ma)
    random.shuffle(train)

Epoch 0 Loss 0.3577238227463844 Acc 0.8453032523592714
Epoch 10 Loss 0.3585691392808854 Acc 0.8390407310869633
Epoch 20 Loss 0.27741738111790826 Acc 0.896732067573775
Epoch 30 Loss 0.10115193868826249 Acc 0.969631141992521
Epoch 40 Loss 0.009688287996244882 Acc 0.9999289409800682
Epoch 50 Loss 0.0415519876323821 Acc 0.983600320172465
Epoch 60 Loss 0.054384881471194114 Acc 0.9874136834038832
Epoch 70 Loss 0.045912286230596615 Acc 0.9940838024085178
Epoch 80 Loss 0.02079570656470188 Acc 0.9917239253468476
Epoch 90 Loss 0.005779521988234355 Acc 0.9999985838818193
Epoch 100 Loss 0.004128102227181615 Acc 0.9999963174525147
Epoch 110 Loss 0.09741377409208155 Acc 0.9730761900724079
Epoch 120 Loss 0.0033819765849131597 Acc 0.9999400624453558
Epoch 130 Loss 0.0005055673678424009 Acc 0.9999999999999946
Epoch 140 Loss 0.01644081386447104 Acc 0.9968513176139709
Epoch 150 Loss 0.00016596590522660083 Acc 0.9999999999999946
Epoch 160 Loss 0.01730864439500782 Acc 0.9935953077416854
Epoch 170 Loss 0.00

In [21]:
# Evaluate on test set
test_acc=0
n=0
for sample in test:
    test_reviews = np.array([sample['w'] ]).reshape([1,-1,300])
    test_labels  = np.array([sample['y']]).reshape([1,1])
    test_labels = np.array(test_labels).reshape([-1, 1])
    test_acc += sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
    n+=1
acc=test_acc/n 
print("Final accuracy:", acc)


Final accuracy: 0.9320113314447592


In [22]:
sess.close()

### Problem 2.3:  Encode each vocabulary word as a one-hot vector. Train an MLP on the average of the onehot vectors.

#### Build one hot embedding functionality

In [64]:
print("len(english_words): ", len(english_words))
print("english_embedddings.shape: ", english_embedddings.shape)

# Build onehot encoding scheme with an identity matrix
onehot_embeddings = np.identity(len(english_words),dtype=np.float32)
print("onehot_embeddings.shape: ", onehot_embeddings.shape)
#print(np.sum(onehot_embeddings,axis=0))
#print(np.sum(onehot_embeddings,axis=1))

# Create LUT
index = {word: i for i, word in enumerate(english_words)}
print("Size of index dict: ", len(index.keys()))

remove_punct=str.maketrans('','',string.punctuation)

# This function converts a line of our data file into
# a tuple (x, y), where x is 150875-dimensional one-hot representation
# of the words in a review, and y is its label.
def convert_line_to_example_onehot(line):
    # Pull out the first character: that's our label (0 or 1)
    y = int(line[0])
    # Split the line into words using Python's split() function
    words = line[2:].translate(remove_punct).lower().split()
    # Look up the embeddings of each word, ignoring words not
    # in our pretrained vocabulary.
    embeddings = [onehot_embeddings[index[w]] for w in words if w in index]
    # Take the mean of the embeddings
    x = np.mean(np.vstack(embeddings), axis=0)
    return {'x': x, 'y': y, 'w':embeddings}

# Apply the function to each line in the file.
enc = 'utf-8' # This is necessary from within the singularity shell

len(english_words):  150875
english_embedddings.shape:  (150875, 300)
onehot_embeddings.shape:  (150875, 150875)
Size of index dict:  150875


#### Create train/test datasets for MLP

In [65]:
### Choose Dataset
with open("Data/movie-simple.txt", "r", encoding=enc) as f:
    dataset = [convert_line_to_example_onehot(l) for l in f.readlines()]  
#with open("Data/movie-pang02.txt", "r",encoding=enc) as f:
#    dataset = [convert_line_to_example_onehot(l) for l in f.readlines()]

print("Length of Dataset: ",len(dataset))

# Split full dataset into train/test splits
random.shuffle(dataset)
batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

print("# train: ", len(train))
print("# test: ", len(test))

Length of Dataset:  1411
# train:  1000
# test:  411


#### Build MLP

In [66]:
### Configs
num_hidden_L1 = 100
num_hidden_L2 = 100
learning_rate = .01
num_epochs = 2000

# Clear all old tf graphs
tf.reset_default_graph()

# Placeholders for input
X = tf.placeholder(tf.float32, [None, 150875]) # Word embedding size = 150875
y = tf.placeholder(tf.float32, [None, 1]) # Binary classification output: "good" or "bad"

# Three-layer MLP
h1 = tf.layers.dense(X, num_hidden_L1, tf.nn.relu)
h2 = tf.layers.dense(h1, num_hidden_L2, tf.nn.relu)
logits = tf.layers.dense(h2, 1)
probabilities = tf.sigmoid(logits)

# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(logits)), y), tf.float32))

# Training
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Initialization of variables
initialize_all = tf.global_variables_initializer()

#### Train MLP

In [67]:
sess = tf.Session()
sess.run(initialize_all)
for epoch in range(num_epochs):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = [sample['x'] for sample in data]
        labels  = [sample['y'] for sample in data]
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
    if epoch % 10 == 0:
        print("Epoch", epoch, "Loss", l, "Acc", acc)
    random.shuffle(train)

# Evaluate on test set
test_reviews = [sample['x'] for sample in test]
test_labels  = [sample['y'] for sample in test]
test_labels = np.array(test_labels).reshape([-1, 1])
print("Test Review Data Shape: ",np.array(test_reviews).shape)
acc = sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
print("Final test accuracy:", acc)

Epoch 0 Loss 0.6936411 Acc 0.42
Epoch 10 Loss 0.6881182 Acc 0.57
Epoch 20 Loss 0.6828872 Acc 0.6
Epoch 30 Loss 0.6769927 Acc 0.63
Epoch 40 Loss 0.6756306 Acc 0.63
Epoch 50 Loss 0.6797851 Acc 0.58
Epoch 60 Loss 0.68623453 Acc 0.53
Epoch 70 Loss 0.6842681 Acc 0.53
Epoch 80 Loss 0.6786213 Acc 0.57
Epoch 90 Loss 0.68487173 Acc 0.52
Epoch 100 Loss 0.66725016 Acc 0.63
Epoch 110 Loss 0.6832987 Acc 0.52
Epoch 120 Loss 0.67074126 Acc 0.59
Epoch 130 Loss 0.690413 Acc 0.45
Epoch 140 Loss 0.67591757 Acc 0.54
Epoch 150 Loss 0.6781906 Acc 0.52
Epoch 160 Loss 0.6841625 Acc 0.47
Epoch 170 Loss 0.6625603 Acc 0.6
Epoch 180 Loss 0.6673707 Acc 0.55
Epoch 190 Loss 0.6705307 Acc 0.56
Epoch 200 Loss 0.6753393 Acc 0.51
Epoch 210 Loss 0.6619837 Acc 0.55
Epoch 220 Loss 0.66405845 Acc 0.57
Epoch 230 Loss 0.6465087 Acc 0.68
Epoch 240 Loss 0.6541261 Acc 0.59
Epoch 250 Loss 0.66265184 Acc 0.57
Epoch 260 Loss 0.6629913 Acc 0.54
Epoch 270 Loss 0.6474299 Acc 0.58
Epoch 280 Loss 0.63664186 Acc 0.69
Epoch 290 Loss 0.633

In [68]:
sess.close()

### Problem 2.4:  Encode each vocabulary word as a one-hot vector. Train RNN on the one-hot encodings.

#### Create train/test datasets for MLP

In [78]:
### Choose Dataset
with open("Data/movie-simple.txt", "r", encoding=enc) as f:
    dataset = [convert_line_to_example_onehot(l) for l in f.readlines()]  
#with open("Data/movie-pang02.txt", "r",encoding=enc) as f:
#    dataset = [convert_line_to_example_onehot(l) for l in f.readlines()]

print("Length of Dataset: ",len(dataset))

# Split full dataset into train/test splits
random.shuffle(dataset)
batch_size = 1
total_batches = len(dataset) // batch_size
train_batches = 3 * total_batches // 4
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

print("# train: ", len(train))
print("# test: ", len(test))

Length of Dataset:  1411
# train:  1058
# test:  353


#### Build RNN Model

Note: Only running for 4 epochs because training is VERY VERY SLOW!!

In [79]:
# Clear old tf stuff
tf.reset_default_graph()

# Configs
n_steps = None
n_inputs = 150875
n_neurons = 100
num_epochs = 4

# Input placeholders
X= tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y= tf.placeholder(tf.float32, [None, 1])

# Build RNN
basic_cell = tf.contrib.rnn.BasicRNNCell(n_neurons,activation=tf.nn.tanh)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
last_cell_output=outputs[:,-1,:]
y_=tf.layers.dense(last_cell_output,1)

# Loss and metrics
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_, labels=y))
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(y_)), y), tf.float32))

# Training
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

#### Train RNN on one-hot encoded movie data

In [80]:
initialize_all = tf.global_variables_initializer()
sess = tf.Session()
sess.run(initialize_all)
l_ma=.74
acc_ma=.5
for epoch in range(num_epochs):
    for batch in range(train_batches):
        data = train[batch*batch_size:(batch+1)*batch_size]
        reviews = np.array([sample['w'] for sample in data]).reshape([1,-1,150875]) # New dims
        labels  = np.array([sample['y'] for sample in data]).reshape([1,1])
        labels = np.array(labels).reshape([-1, 1])
        _, l, acc = sess.run([train_step, loss, accuracy], feed_dict={X: reviews, y: labels})
        l_ma=.99*l_ma+(.01)*l
        acc_ma=.99*acc_ma+(.01)*acc
        #if (batch+1) % 100 == 0:
        #    print("batch", batch, "Loss", l_ma, "Acc", acc_ma)
    if epoch % 1 == 0:
        print("Epoch", epoch, "Loss", l_ma, "Acc", acc_ma)
    random.shuffle(train)

Epoch 0 Loss 0.4587483599366612 Acc 0.7884598111950616
Epoch 1 Loss 0.16320897128847828 Acc 0.9130764524316867
Epoch 2 Loss 0.07306027829956457 Acc 0.9831512704504691
Epoch 3 Loss 0.10277702489273653 Acc 0.9690887780244798


#### Test trained RNN on one-hot encoded test data

In [81]:
# Evaluate on test set
test_acc=0
n=0
for sample in test:
    test_reviews = np.array([sample['w'] ]).reshape([1,-1,150875])
    test_labels  = np.array([sample['y']]).reshape([1,1])
    test_labels = np.array(test_labels).reshape([-1, 1])
    test_acc += sess.run(accuracy, feed_dict={X: test_reviews, y: test_labels})
    n+=1
acc=test_acc/n 
print("Final test accuracy:", acc)

Final test accuracy: 0.9320113314447592


In [82]:
sess.close()

### Problem 2.5: Why did the word embeddings work better (hint: the word embeddings will work better…)

The word embeddings worked better because they are lower dimensions and it is intuitively simpler to learn the same task with less dimensions. It is favorable to have a higher ratio of training examples to number of input dimensions. In the case of the word embeddings the ratio is 1000 examples to 300 dimensions ($ratio>1$), while in the one hot embeddings the ratio is 1000 examples to 150875 dimensions ($ratio<1$). If we compare to Imagenet task, there are over 1,000,000 examples with about 50,000 dimensions, so $ratio>1$.

### Problem 2.6: How does cross-validation change when considering a time-series instead of multiple instances (as in our movie reviews)? Only a description is needed.

In time series data, we may do cross-validation on data from the same sample. However, we may not do this with multiple instances. For example, we can split the time series data from a single sample into distinct parts and train on some and validate on others. If we have 3 seconds of audio data, we may train on the first two seconds and validate on the last second. Clearly, this will not work for multiple instances because we treat each instance as a distinct unit, not to be split up.

We can use movie reviews as another example. With time series data we may only consider part of a movie review (i.e. 10 words) for training, and another part (i.e. 8 words) of that same review for validation. If the data is not time series, we must train and validate on data from completely different reviews.

### Problem 2.7: In our previous homework assignment we considered the conditional GAN. In that case, the conditional label was known and given. Instead, consider generating images to match text. One approach could be to use an RNN to encode text to a vector that is fed to a conditional GAN (e.g. http://proceedings.mlr.press/v48/reed16.pdf). Draw a graph (but do not implement) how such a system could work. Any implementation here is completely optional, we are only looking for a description of how this could work.

Refer to "WrittenQuestions" submission.